### Import Modules

In [2]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

matplotlib.rcParams['figure.figsize'] = [10, 10]

In [49]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [50]:
p_dict = {
    'order' : {
        'fit' : 'nlo', # 'nlo', 'nnlo', or 'nnnlo'
        'latt_spacing' : 2, # no order 1 term -- starts at 2
        'vol' : 8 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'simultaneous', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    
    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_prior' : False,
    'save_results' : False,
}

### Do fit

In [51]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print bootstrapper
    
if p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Making fits...
100% complete Time (s):  54.5550000668
Compiling results...
Done.

Total time (s):  60.5080001354 



In [53]:
#print bootstrapper.fits[0]

### Or make all fits

In [41]:
p_dict = {
    'order' : {
        'latt_spacing' : 2, # no order 1 term -- starts at 2
        'vol' : 8 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'

    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],

    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,

    'save_prior' : False,
    'save_results' : True,
}

In [42]:
for fit_type in ['xpt', 'xpt-taylor', 'ma', 'ma-taylor']:
    for order_fit in ['nlo', 'nnlo', 'nnnlo']:
        
        print fit_type, order_fit
        
        # Specify fit parameters
        p_dict['order']['fit'] = order_fit
        p_dict['fit_type'] = fit_type
        
        # Do fit
        t0 = time.time()

        # Load data
        data_loader = dl.data_loader()
        fit_data = data_loader.get_fit_data()

        # Get prior
        prior = None
        if p_dict['use_prior']:
            prior = data_loader.get_prior(p_dict['fit_type'])

        # Make bootstrapper
        bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                                       fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

        if p_dict['make_plots']:
            data_loader.save_plots(
                bootstrapper.make_plots(
                    show_error_ellipses=p_dict['show_error_ellipses'], 
                    show_bootstrap_histograms=p_dict['show_bs_histograms']),
                output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
            )
        else:
            print bootstrapper

        if p_dict['save_prior']:
            print 'Saving prior...'
            data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])

        if p_dict['save_results']:
            data_loader.save_fit_info(bootstrapper.get_fit_info())


        t1 = time.time()

        print "\nTotal time (s): ", t1 - t0, "\n"

xpt nlo
Making fits...
100% complete Time (s):  0.31299996376
Compiling results...
Done.
Saving...
Done.

Total time (s):  6.02500009537 

xpt nnlo
Making fits...
100% complete Time (s):  0.679999828339
Compiling results...
Done.
Saving...
Done.

Total time (s):  6.52299976349 

xpt nnnlo
Making fits...
100% complete Time (s):  0.827000141144
Compiling results...
Done.
Saving...
Done.

Total time (s):  6.89100003242 

xpt-taylor nlo
Using default prior.
Making fits...
100% complete Time (s):  0.543999910355
Compiling results...
Done.
Saving...
Done.

Total time (s):  6.23199987411 

xpt-taylor nnlo
Using default prior.
Making fits...
100% complete Time (s):  0.492000102997
Compiling results...
Done.
Saving...
Done.

Total time (s):  6.41599988937 

xpt-taylor nnnlo
Using default prior.
Making fits...
100% complete Time (s):  0.455000162125
Compiling results...
Done.
Saving...
Done.

Total time (s):  6.33699989319 

ma nlo
Making fits...
100% complete Time (s):  1.09700012207
Compiling 

In [92]:
#bootstrapper.plot_parameter_histogram('L_5')

In [86]:
#print bootstrapper

In [87]:
#print bootstrapper.fits[0]

In [220]:
#print bootstrapper.fits[0]

In [214]:
#bootstrapper.fits[0].show_plots()

In [34]:
#print bootstrapper.fits[0]